<a href="https://colab.research.google.com/github/yoshitoG/nicar21_first_python_notebook/blob/main/%E3%80%90%E8%AA%B2%E5%A4%96%E8%A3%9C%E8%AC%9B0710%E3%80%91%E3%83%AF%E3%82%AF%E3%83%81%E3%83%B3%E6%8E%A5%E7%A8%AE%E3%81%A8%E6%96%B0%E8%A6%8F%E6%84%9F%E6%9F%93%E8%80%85%E6%95%B0%E3%81%AE%E9%96%A2%E4%BF%82%E3%82%92%E8%A6%8B%E3%81%9F%E3%81%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 新型コロナワクチン接種と、感染者数の関係を見たい
### Google Colab で「R」を試す

小宮山さんや山本さんの講義の補足。終了後にゆっくり目を通していただければ十分です。Slackのinformationチャンネルに松波さんが投稿している、tidyverseの基本の[解説](https://jnpcitseminar.slack.com/files/UGX4CGHNZ/F026EJU680M/memo_for_r_beginners.rmd)へのつなぎ、だと思って下さい。

このノートのコードは、PythonではなくRです。なお、Google Colabolatory で実行環境をRにしたノートを新規作成するには、[こちらのリンク](https://colab.research.google.com/notebook#create=true&language=r)を使うのが手っ取り早い方法です。

Colabで見ていただいている方は、上のメニュー欄の「ファイル」から「ドライブにコピーを保存」を選んだ後、ご自身の環境で実際に動かして試してみて下さい。

お急ぎの方は、２の「その前に」は飛ばして、３の「使い始めの設定」から実行して下さい。コードはローカルでも動きます。

## 1.何をやるか

### 1.1. 時系列で可視化したい

[政府CIOポータル](https://cio.go.jp/c19vaccine_dashboard)から、COVID-19のワクチン接種状況のデータを取ってくるところまでは、同じです。さらに、別の場所から、感染状況のデータを取ってきて、両者の関係を見てみることにします。感染状況は、[内閣官房「新型コロナウイルス感染症対策」](https://corona.go.jp/dashboard/)のデータのページにまとまっています。

ざっと作っただけですが、[こちら](https://bit.ly/JNPC210710)のように、時間の経過でどう変化しているかを可視化する、という想定で、データを加工していきましょう。接種のほうは件数の累計が、感染者数のほうは新規の移動平均が、最終的にほしいデータです。

なお、可視化に使ったのは、[Flourish](https://flourish.studio/)です。Flourishについては説明しません。

### 1.2. 気になる点

データの概要を見てみましょう。接種状況のほうは、**その日その日の件数**です。都道府県別になっています。

一方、感染状況は、**累積の感染者数**ですね。こちらも都道府県別になっている（全国計のデータは別になっている）のですが、**自治体名ではなくて数字**が並んでいます。

ちなみに、接種完了の人のstatusは単に「2」とされていて、2回必要なうち2回目まで受けたのか、1回だけでよいワクチンなのかが、区別できません。ワクチンの効果を今後、検証していくためには、「1回で完了」「2回目中の1回目＝途中」「2回目中の2回目＝完了」の区別がつくよう、「1-1」「2-1」「2-2」のような区分にしておいてほしかったところです。もしかしたら、変更になるかもしれませんけれど。

<font color="gray">
現状では1回完了型は日本では接種できないと思います（違っていたらごめんなさい）が、仕様を見ると、1回完了型の場合もstatusは2とするようです。
</font>

ワクチンの製造元別に数字が分かれば、なおよいですよね。

データをもらいっぱなしにするのではなく、使う側として、声を上げていかなくてはいけない部分だと思います。

## 2.その前に


### 2.1. わざわざプログラムを書く理由

ExcelやGoogleスプレッドシートがあるのに、どうしてプログラムを書くのか。一番の理由は、プログラムに書いておけば、**繰り返し使えて、再現可能性が高い**から、です。

Excelに代表される表計算ソフトは、優れた発明です。試行錯誤しながら処理していくのにも、向いています。しかし、たとえて言うなら「食材」（データ）と「レシピ」（処理）が渾然一体となっているという欠点があります。自分が作ったものでも、どこにどんな式を入れたのだったか、分からなくなってしまうことが、よくあります。

RやPythonのプログラムであれば、データと処理が分離されています。その分、見通しがよく、何をやろうとしているかが分かる、ということになります。なので、必要な手直しもしやすいし、データが更新されても同じ処理が適用できると分かれば、同じプログラムを再度走らせて、最新の結果を得ることもできます。

### 2.2. プログラムは人生ゲーム

暗号のように見えるプログラムですが、一部分を取り出して見てみれば、やっていることはシンプルです。「人生ゲーム」のように、道に枝分かれのある双六みたいなものです。

* 変数と計算　　　手持ち資金や、車に乗っている人数など、今の状態を記憶させおく「変数（入れ物）」を使って、計算をする
* 条件判定と分岐　分かれ道。何かを調べて、処理方法を変える。ルーレットを回すように、乱数でランダムに決めることも（サンプリングなど）
* 繰り返し　　　　必要なだけ反復処理する。PCは疲れません

途中の道順やスピードはさまざまでも、最終目的地にたどり着けるなら、プログラムとしては役に立ちます。手直しのしやすさや、効率に差はあるにしても。

傾向として、Rは書く人によって、プログラムが大きく異なります。選べる道が多いのはよいけれど、その分、迷いやすい。Pythonは、誰が書いても、似たプログラムになります。主要道があらかじめ整備されているかわり、そこしか通れない（抜け道はあまりない）感じで、これは大きなメリットです。

### 2.3. プログラムのお約束

変数に何かを入れて、覚えておいてもらうことを、代入といいます。これに使う記号が、歴史的経緯で「=」なのです（Rでは「<-」も使えます）。分かりにくいのですが、イコールではなく、**右辺のものを左辺の変数に入れる**、という指示になります。

In [1]:
born = 1984         # 1984年生まれの人が
age  = 37           # 誕生日を迎えて37歳だとすると
now  = born + age   # 今は何年？
print(now)

[1] 2021


変数やデータには型があって、引用符で囲んでないものは数値または変数の中身、引用符に囲まれているものは「そのままの文字列（リテラル）」です。1984は数値、"1984"は文字列です。

In [2]:
born2 = "1984"
age2  = "37"

# エラーで止まらないようにしたければ、次のセルの冒頭に「#」を入れる。
# print(born2 + age2)
# のように、注釈にすること

In [3]:
# print(born2 + age2)

Rは「文字列の足し算はできません」と、エラーメッセージで教えてくれました。このあたりの挙動は、言語の設計によって違います。エラーしないで、計算結果が"198437"という文字列になるプログラム言語もあります。

In [4]:
# ある変数を最初に0にしておき
counter = 0
print(counter)
# 繰り返し処理をするごとに1を足すなら
counter = counter + 1
print(counter)
# カウンターとして使える

[1] 0
[1] 1


このコードのしていること、想像がつきますか？　ある数「counter」に１を足したら自分自身に等しくなる、という**不能方程式ではありません**。これは、行番号5のところで、指を折るようにして回数を記録していく「カウンター」です。

では、「イコール」の意味のときはどう書くか。「==」と重ねて使います。

In [5]:
born == age

[1] FALSE

1985と37は等しくありませんよね。

ここでうっかり、等号ひとつだけを使ってしまうと、どうなるでしょう？

In [6]:
born = age
print(born)

[1] 37


変数bornに、変数ageの中身である37が代入されてしまいました。

では、これはどうなるでしょう？

In [7]:
print(age)  # こちらは数値。
print(age2) # 引用符がついているのは、数値でなく文字列だから。
age == age2

[1] 37
[1] "37"


[1] TRUE

片方は数値、片方は文字列なのに、「等しい」と判定されました。これは、Rの仕様のようですが、善し悪しですね。

見た目は一緒でも、型が違えば「等しくない」と厳密に判定する記号「===」が使える言語もあります。気を利かせて型を変換してしまう言語もあります。

### 2.4. ファンクション（関数）とは

関数、という名前がものものしく、数学を連想させるせいで身構えてしまいますが、要するに「ファンクション」とは機能のことです。パソコンについている「F1」キーを「関数キー」だと思う人はいませんよね。ヘルプを表示する、という働きをひとまとめにして、このキーに対応させているだけです。ある「働き」とか「機能」「処理」だと考えて下さい。

<font color = 'gray'>
※函数や関数も名訳だし、プログラムに当てはめても、別におかしくはありませんので、アレルギーのない方は無視して下さい。
</font>

さて、Mというデータを、まずAという処理をして、次にBする場合、数学的あるいはExcel的な書き方では、こうなります。中から外へ、処理が実行されます。

```
B(A(M))
```

PythonやRでは、上の書き方も使えますが、Rで標準的なのは

```
M %>% A() %>% B()
```

です。()の中に、一つ前の結果を入れていくわけです。左から右へ読んでいけば、そのまま処理の順番になりますね。見慣れない記号は、「パイプ」といいます。RStudioを使っていれば、簡単に入力できます。

```
M %>%
A() %>%
B()
```

のように改行して書いてあっても、その間に注釈（人間用のメモ。PCは実行せずに飛ばします）が書いてあっても、同じように動きます。

Rの場合、これまでも出てきているように、「#」から行末までが注釈として扱われます。

### 2.5. 拡張パッケージ

本日、RやPythonを使うときに、「tidyverse」とか「pandas」という拡張パッケージを入れていました。私も、わざわざどうして？と、初めてRに触ったときは不思議でした。

理由は後から分かってきたのですが、いまやこのどちらも、スプレッドシートやデータベース的なこと（データ処理や科学技術計算）をやるときには、必需品だから、です。

乱暴に言えば、素のままのRやPythonを、表形式のデータを一貫した覚えやすいコードで扱えるよう、チューニングしている、ということです。今から勉強する人は、わざわざ旧式の、分かりにくい処理を覚える必要はありませんから、この２つに慣れて下さい。素のままでなくて、こちらのほうが、デファクトスタンダードです。

## 3.使い始めの設定

素のままのRに入っていない拡張パッケージを入れます。一度だけ`install.packages`でインストールし、あとは`library`または`require`で呼び込むのが普通の書き方です。

ただし今は、Google Colab上なので、何度も読み込みが発生しないよう、ちょっと工夫しました。

JSONデータの読み込みのほか、移動平均を計算したり、日付を扱ったりするためのパッケージ追加しています。

In [8]:
# 毎回、全パッケージをインストールすると時間がかかる上、プログラムも長くなるので、工夫した書き方がこちら。特殊なので、覚える必要はない。
# ローカルなら、インストールは一度だけ。オンラインでも、セッションが継続していたら、不要。なので……
# pacmanパッケージが入ってなかったら、まずそれだけ入れる。この下、行番号4のコードは、「p_boot」を実行して、クリップボードから貼り付けられる。
if (!require("pacman")) install.packages("pacman")
# あとはpacmanにやらせる。ちなみに、名前は「パッケージマネージャー」の意。これを入れてもゲームはできない。
pacman::p_load(tidyverse, jsonlite, ndjson, lubridate, RcppRoll)

Loading required package: pacman

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘pacman’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)


ndjson installed

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)


RcppRoll installed



## 4.接種状況

### 4.1. データを読み込む

ワクチン接種状況のデータを読み込みます。ここは、小宮山さんの講義の通りのやり方です。

In [9]:
# データのありかは以下。
target_url_cio <- "https://vrs-data.cio.go.jp/vaccination/opendata/latest/prefecture.ndjson"
# 有効なndjson形式かを確かめる。ここで使うvalidateは、ndjsonパッケージのほう。
# 読み込みに使うjsonliteにも同名の「validate」があり、衝突するので、パッケージ名を明示する。
ndjson::validate(target_url_cio, verbose = TRUE)
# それを読み込む。
cio_stream <- jsonlite::stream_in(gzcon(url(target_url_cio)))
# 冒頭を表示して確かめる。
head(cio_stream)

File: https://vrs-data.cio.go.jp/vaccination/opendata/latest/prefecture.ndjson


[1] TRUE

opening gzcon input connection.



 Imported 35263 records. Simplifying...


closing gzcon input connection.



,date,prefecture,gender,age,medical_worker,status,count
,<chr>,<chr>,<chr>,<chr>,<lgl>,<int>,<int>
1,2021-04-12,01,F,-64,FALSE,1,113
2,2021-04-12,01,F,65-,FALSE,1,88
3,2021-04-12,01,M,-64,FALSE,1,39
4,2021-04-12,01,M,65-,FALSE,1,26
5,2021-04-12,01,U,-64,FALSE,1,12
6,2021-04-12,01,U,UNK,FALSE,1,1


### 4.2. 型を変換する

スプレッドシートと同様の二次元の表形式。Rにはこれが2種類あります。従来型の「data.frame」になっているので、現在の主流で使いやすい「tibble」に直します。

tibbleやfata.frameには、縦のカラムは、異なる型のデータが混在してはいけない（１列全部、同じデータ型でないといけない）、という大事な制約があります。

縦と横は、表計算ソフトではあまり気にしない点かもしれません。しかし、RやPythonでデータ処理をする際には、縦には型が揃っている、という点はとても重要です。対して、横に見ていくならば、ある１つのモノ・コトについてのさまざまな側面が分かる、という構造です。

<font color = 'gray'>
※底流には、タイプライター以来の横書きのカルチャーがありそうです。
</font>

型は、この上の表の見出し部分、カラム名の下に表示されています。例えば、＜CHR＞は文字列型、＜lgl＞は真と偽の二値を取る論理型です。

dateのカラムは、日付型に直しましょう。このデータではたまたま、年と月と日の判別ができますが、そうでないときもあるので、lubridateパッケージのymdで「年月日の順だよ」と指定しています。

あと、好みの問題もありますが、何種類かの決まった値しか出てこないはずのデータは、「カテゴリー型（因子型）」にしておくと、おかしな値が入っていた場合に気づきやすくなります。「島取」や「鳥根」が紛れ込んでいた場合を考えてみて下さい。

サイトにあったデータの仕様表によれば、genderやageは、今のところ3種類に分類するために使っているようです。

都道府県コードも、0から始まっていますし、厳密に言えば数値（比率尺度・間隔尺度のデータ）ではありません。並び順、という意味合いはありますが、04の宮城が02の青森の2倍、ということはありません。背番号みたいなものですね。

カテゴリー型は、文字列のように見えますが、Rの内部では「数字」＋「名札」です。「山田さん」という人名の文字列を、「出席番号××番」という番号と、「××番の人は山田さん」という対照表にしているのです。データに出てくるのはあるクラスの人に限られる、という状況だと、このほうが設計しやすいことがあります。

見た目では文字列と区別しにくいのでご注意。Rの画面では、引用符付きの「"山田"」なら文字列、引用符に囲まれない「山田」ならカテゴリー型、と区別して表示されます。

In [10]:
# 実は二次元の表にも２種類ある。
# 標準はdata.frameという表形式。でも、tibbleという形式のほうが何かと扱いやすいので、これに変換する。
cio_data <- cio_stream %>%
    as_tibble()
# ここでいったん、結果を表示。冒頭のみ。
head(cio_data)

# 日付型に直したいカラム
columns_to_date <- "date"
# 因子型（カテゴリー型）に直したいカラム。直さずに文字列型でも、まあいい。
columns_to_factor <- c("gender", "age")
# genderは（男尊女卑でなく並べ替えの練習として）、よくある男・女・不明の順にする指定を入れることも可能
gender_levels <- c("M", "F" ,"U")
# ワクチン接種回数は整数型
columns_to_integer <- "count"
# 変な値があったら気がつくようにしたい。readrを使う。
cio_data <- cio_data %>% 
    mutate(
# lubridateのymdを使ったのは、とくに月と日の順番を明示的に指定するため。年は4桁になっているが、07-10が7月10日と10月7日のどちらを意味するか、を指示したほうが安全。
    across(all_of(columns_to_date), lubridate::ymd),
    across(all_of(columns_to_factor), as_factor),
    across(all_of(columns_to_integer), as.integer)
    )
cio_data$gender <- cio_data$gender %>%
    fct_relevel(gender_levels)

# もう一度、処理結果を表示。北海道と青森の全カテゴリーを見たいので、表示を増やした。
head(cio_data, n = 15)
# 北海道の4月12日は合計してみると279件のようだ。青森は203。
# 沖縄の直近のデータも見てみる。
tail(cio_data, n = 15)
str(cio_data)
summary(cio_data)
# 不要になった元データは消す
# cio_data %>% is.na() %>% sum()
rm(cio_stream)

date,prefecture,gender,age,medical_worker,status,count
<chr>,<chr>,<chr>,<chr>,<lgl>,<int>,<int>
2021-04-12,01,F,-64,FALSE,1,113
2021-04-12,01,F,65-,FALSE,1,88
2021-04-12,01,M,-64,FALSE,1,39
2021-04-12,01,M,65-,FALSE,1,26
2021-04-12,01,U,-64,FALSE,1,12
2021-04-12,01,U,UNK,FALSE,1,1


date,prefecture,gender,age,medical_worker,status,count
<date>,<chr>,<fct>,<fct>,<lgl>,<int>,<int>
2021-04-12,01,F,-64,FALSE,1,113
2021-04-12,01,F,65-,FALSE,1,88
2021-04-12,01,M,-64,FALSE,1,39
2021-04-12,01,M,65-,FALSE,1,26
2021-04-12,01,U,-64,FALSE,1,12
2021-04-12,01,U,UNK,FALSE,1,1
2021-04-12,02,F,-64,FALSE,1,3
2021-04-12,02,F,65-,FALSE,1,137
2021-04-12,02,M,-64,FALSE,1,2


date,prefecture,gender,age,medical_worker,status,count
<date>,<chr>,<fct>,<fct>,<lgl>,<int>,<int>
2021-07-09,46,M,-64,FALSE,2,97
2021-07-09,46,M,65-,FALSE,1,1211
2021-07-09,46,M,65-,FALSE,2,1571
2021-07-09,46,U,UNK,FALSE,1,5
2021-07-09,46,U,UNK,FALSE,2,1
2021-07-09,47,F,-64,FALSE,1,599
2021-07-09,47,F,-64,FALSE,2,341
2021-07-09,47,F,65-,FALSE,1,719
2021-07-09,47,F,65-,FALSE,2,1776


tibble [35,263 × 7] (S3: tbl_df/tbl/data.frame)
 $ date          : Date[1:35263], format: "2021-04-12" "2021-04-12" ...
 $ prefecture    : chr [1:35263] "01" "01" "01" "01" ...
 $ gender        : Factor w/ 3 levels "M","F","U": 2 2 1 1 3 3 2 2 1 1 ...
 $ age           : Factor w/ 3 levels "-64","65-","UNK": 1 2 1 2 1 3 1 2 1 2 ...
 $ medical_worker: logi [1:35263] FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ status        : int [1:35263] 1 1 1 1 1 1 1 1 1 1 ...
 $ count         : int [1:35263] 113 88 39 26 12 1 3 137 2 54 ...


      date             prefecture        gender     age        medical_worker 
 Min.   :2021-04-12   Length:35263       M:14112   -64:14114   Mode :logical  
 1st Qu.:2021-05-14   Class :character   F:14227   65-:14671   FALSE:35263    
 Median :2021-06-02   Mode  :character   U: 6924   UNK: 6478                  
 Mean   :2021-06-01                                                           
 3rd Qu.:2021-06-21                                                           
 Max.   :2021-07-09                                                           
     status          count      
 Min.   :1.000   Min.   :    1  
 1st Qu.:1.000   1st Qu.:   25  
 Median :1.000   Median :  145  
 Mean   :1.421   Mean   : 1338  
 3rd Qu.:2.000   3rd Qu.: 1108  
 Max.   :2.000   Max.   :36359  

### 4.3. カテゴリー別の数字を合算する
ワクチン接種状況の数字は、その日その日のカテゴリー別です。年齢・性別・医療従事者かどうかに関わらず、合算することにしましょう。

現時点では、医療従事者フラグは準備されているだけです。でも、将来的には入手可能になることも見越したデータ構造にしてくれているのが、嬉しいですね。製造元の欄も確保されていれば、なおよいのだけれどなあ。

さて、合算は、このままではやりにくいので、縦長の表から横長の表にいったん変換します。ある県のある日の分が横に並ぶようにして、その合計を計算したカラムを新しく作ります。さらに後ほど、その日までの累計も計算します。

縦横変換しなくても、やり方次第で、県ごとにある日の分を合計することは可能です。ただ、将来的にstatusや医療従事者フラグによって重み付けを変えて計算したい場合には、横持ちの表にしておいたほうが対応しやすそうなので、この方法を選択しました。

In [11]:
cio_wide <- cio_data %>%
# 属性別の接種回数をひとつにまとめる準備。
    select(date, prefecture, status, everything()) %>% 
# 縦持ちから、横持ちに変形する。
    pivot_wider(
        names_from = status:medical_worker,
        values_from = count
    )
# こんな表になった
head(cio_wide)

# ある県の１日あたりの接種を合算。１回目か２回目か、１回で完了かは区別せずに単純合算。
vaccinations <- cio_wide %>% 
    transmute(
# date,prefectureのカラムはそのまま残す。
        date,
        prefecture,
# 各カテゴリーの接種回数を合算したカラムを作る。
        n_vaccinations  = select(., contains("_")) %>%
        rowSums(na.rm = TRUE)
    )

# ここで結果を見てみる。初日の北海道は279件、青森県は203件。
head(vaccinations)
# 沖縄の直近の日付も合っているようだ。
tail(vaccinations)
summary(vaccinations)

# 不要なデータを消す。
rm(cio_data, cio_wide)

date,prefecture,1_F_-64_FALSE,1_F_65-_FALSE,1_M_-64_FALSE,1_M_65-_FALSE,1_U_-64_FALSE,1_U_UNK_FALSE,1_U_65-_FALSE,1_F_UNK_FALSE,2_F_65-_FALSE,2_M_65-_FALSE,2_F_-64_FALSE,2_M_-64_FALSE,2_U_UNK_FALSE,2_U_65-_FALSE,2_U_-64_FALSE,2_F_UNK_FALSE,1_M_UNK_FALSE,2_M_UNK_FALSE
<date>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
2021-04-12,01,113,88,39,26,12,1,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2021-04-12,02,3,137,2,54,NA,7,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2021-04-12,03,NA,23,NA,27,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2021-04-12,04,1,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2021-04-12,05,13,4,3,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2021-04-12,06,10,23,3,17,NA,1,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


date,prefecture,n_vaccinations
<date>,<chr>,<dbl>
2021-04-12,01,279
2021-04-12,02,203
2021-04-12,03,50
2021-04-12,04,1
2021-04-12,05,20
2021-04-12,06,54


date,prefecture,n_vaccinations
<date>,<chr>,<dbl>
2021-07-09,42,5754
2021-07-09,43,10146
2021-07-09,44,4865
2021-07-09,45,7357
2021-07-09,46,8151
2021-07-09,47,6357


      date             prefecture        n_vaccinations  
 Min.   :2021-04-12   Length:4148        Min.   :     1  
 1st Qu.:2021-05-04   Class :character   1st Qu.:  1100  
 Median :2021-05-26   Mode  :character   Median :  6039  
 Mean   :2021-05-26                      Mean   : 11378  
 3rd Qu.:2021-06-17                      3rd Qu.: 13712  
 Max.   :2021-07-09                      Max.   :119822  

### 4.4. 全国計を計算する
全国計は、接種件数の場合は別にデータがあるわけではなく、各都道府県の数字を合算すればよさそうです。感染者数とは違い、客船のようなケースはないからでしょうか。

<font color = "gray">
大規模接種の場合も、きちんんと居住地に振り分けられているのかどうか、要確認ですね。
</font>

なお、全国計も、縦のまま計算することは可能です。

In [12]:
# いったん横持ちの表にする。
vaccinations <- vaccinations %>%
    pivot_wider(
        names_from = prefecture,
        values_from = n_vaccinations
    ) %>% 
# 全国計のカラム"TOTAL/AVE"を作り、他の列の合計を入れる。
    mutate(
        "TOTAL/AVE" = select(., -date) %>%
        rowSums(na.rm = TRUE)
    ) 
# こんな表になった
head(vaccinations)

vaccinations <- vaccinations %>% 
# 縦持ちの表に戻す。
    pivot_longer(
        cols = -date,
        names_to = "prefecture",
        values_to = "n_vaccinations",
        values_drop_na = TRUE
    )

# 結果を確認する。
summary(vaccinations)
head(vaccinations)
tail(vaccinations)

date,01,02,03,04,05,06,07,08,09,⋯,43,44,45,46,47,17,34,19,31,TOTAL/AVE
<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2021-04-12,279,203,50,1,20,54,201,61,10,⋯,55,5,54,22,13,NA,NA,NA,NA,4737
2021-04-13,301,380,35,15,27,31,159,54,15,⋯,71,13,48,18,5,70,16,NA,NA,4112
2021-04-14,261,150,95,3,232,62,272,57,11,⋯,34,11,33,26,52,NA,11,2,1,4794
2021-04-15,740,315,99,44,326,133,201,217,15,⋯,71,154,135,18,167,99,18,122,59,8492
2021-04-16,919,283,84,42,131,147,173,597,14,⋯,79,70,126,123,35,29,64,42,20,9796
2021-04-17,311,118,72,19,135,1,NA,174,43,⋯,135,41,19,196,31,235,98,126,77,7652


      date             prefecture        n_vaccinations   
 Min.   :2021-04-12   Length:4237        Min.   :      1  
 1st Qu.:2021-05-04   Class :character   1st Qu.:   1146  
 Median :2021-05-26   Mode  :character   Median :   6293  
 Mean   :2021-05-26                      Mean   :  22279  
 3rd Qu.:2021-06-17                      3rd Qu.:  14418  
 Max.   :2021-07-09                      Max.   :1214755  

date,prefecture,n_vaccinations
<date>,<chr>,<dbl>
2021-04-12,01,279
2021-04-12,02,203
2021-04-12,03,50
2021-04-12,04,1
2021-04-12,05,20
2021-04-12,06,54


date,prefecture,n_vaccinations
<date>,<chr>,<dbl>
2021-07-09,47,6357
2021-07-09,17,8304
2021-07-09,34,20903
2021-07-09,19,4166
2021-07-09,31,3989
2021-07-09,TOTAL/AVE,724128


### 4.5. 累計を計算する

ワクチンが行き渡ることの効果は、その日の接種分ではなく、それまでの積み重ねによるものでしょうから、累計の数字を計算しましょう。特定の都道府県に注目して、その県で接種が始まった日からの件数を足していく、ということになります。

ある県だけに注目したら、こういう処理をしたらいいね、という部分を、即席のファンクションにしています。この部分は、informationチャンネルの[解説「Memo_for_R_beginners.Rmd」](https://jnpcitseminar.slack.com/files/UGX4CGHNZ/F026EJU680M/memo_for_r_beginners.rmd)を参考にして下さい。

もともとは、当日分を含む累計しか計算していなかったのですが、積み上げ棒グラフにするなら、前日までの累計が必要だな、と考えて後からその部分を追加しました。ファンクションとして、切り出してまとめていたから、手直しが楽でした。

本当は、所定回数を完了したのかまだ途中かを区別し、接種の効果に応じてカウントの重みを変えたいところです。しかし、１回完了型が接種可能になった際、いま入手できるデータでは対応しきれないため、単純に合算しています。「2回中の1回」（statusが1）は判別できるのですが、statusが2の場合、1だった人が2回目接種を終えたのか、1回だけでＯＫのワクチンの接種を受けたのかが、区別できないためです。

製造元がどこか（Johnson & Johonson とか）の情報があれば、補えます。流通の目詰まりを追うためにも、データがほしいところです。

In [13]:
# 都道府県ごとに小分けした想定で、累計を計算する即席のファンクションを作った。
calc_sum = function(df){
    return(
        df %>% arrange(date) %>% 
            mutate(
# total_vaccinationsは当日分も含む。
                total_vaccinations = cumsum(n_vaccinations),
# 当日分と積み上げグラフにする場合も考えて、前日分まではcum_vaccinationsカラムに入れることにした。
                cum_vaccinations   = total_vaccinations - n_vaccinations
            ) %>% 
        arrange(desc(date))
    )
}

# 都道府県ごとにnest（下の階層に畳む）する。
vaccinations <- vaccinations %>%
# コード順にソートしておく。
    arrange(prefecture) %>%
# prefecture以外は畳む。
    nest(data_by_pref = -prefecture) %>%
# さっき作ったファンクションを、県単位で当てはめていく。
    mutate(data_by_pref = map(data_by_pref, calc_sum)) %>% 
# 計算が終わったので、nestを解除する。
    unnest(cols = data_by_pref)

# 表示する。
head(vaccinations)
tail(vaccinations)
str(vaccinations)

prefecture,date,n_vaccinations,total_vaccinations,cum_vaccinations
<chr>,<date>,<dbl>,<dbl>,<dbl>
01,2021-07-09,20416,1901266,1880850
01,2021-07-08,33336,1880850,1847514
01,2021-07-07,36208,1847514,1811306
01,2021-07-06,40353,1811306,1770953
01,2021-07-05,44455,1770953,1726498
01,2021-07-04,37891,1726498,1688607


prefecture,date,n_vaccinations,total_vaccinations,cum_vaccinations
<chr>,<date>,<dbl>,<dbl>,<dbl>
TOTAL/AVE,2021-04-17,7652,39583,31931
TOTAL/AVE,2021-04-16,9796,31931,22135
TOTAL/AVE,2021-04-15,8492,22135,13643
TOTAL/AVE,2021-04-14,4794,13643,8849
TOTAL/AVE,2021-04-13,4112,8849,4737
TOTAL/AVE,2021-04-12,4737,4737,0


tibble [4,237 × 5] (S3: tbl_df/tbl/data.frame)
 $ prefecture        : chr [1:4237] "01" "01" "01" "01" ...
 $ date              : Date[1:4237], format: "2021-07-09" "2021-07-08" ...
 $ n_vaccinations    : num [1:4237] 20416 33336 36208 40353 44455 ...
 $ total_vaccinations: num [1:4237] 1901266 1880850 1847514 1811306 1770953 ...
 $ cum_vaccinations  : num [1:4237] 1880850 1847514 1811306 1770953 1726498 ...


## 5.データの橋渡し

### 5.1. 何が足りない

冒頭1.2.の「気になる点」で触れた通り、
ワクチン接種と感染状況のデータの作りは、似ているようで違っています。関係を見るためには、両方のデータを、同じ日付・同じ都道府県、でドッキングさせたい。となると、都道府県コードと都道府県名との対照表が必要になります。

01と北海道は同じもの、02と青森または青森県も同じもの……と、重ならず抜け漏れもなく対応させるためです。

### 5.2. ネ申（ねもうす）Excel問題

仕様のところに、JIS規格へのリンクも張ってあるのですが、ご覧になった方はいるでしょうか？　印刷したときに紙1枚に納まるよう、人間にやさしく作ってありました。が、PCに処理させるには不向きです。
参考になる、三重大・奥村先生の資料は[こちら](https://oku.edu.mie-u.ac.jp/~okumura/SSS2013slide.pdf)。山本さんの資料にも、参考になるリンクが紹介されていました。

仕方がないので、Rで扱いやすいよう、都道府県コードだけで1カラム、県名だけで1カラム、になったデータを手元で作りました（表計算ソフトで作ったあと、読み込ませたもの）。

あと、生数字ではなく、人口比で見るようにしたいので、2021年時点の推計人口（単位・千人）のデータも用意しました。出所は[こちら](https://www.stat.go.jp/data/nihon/zuhyou/n210200300.xlsx)。

<font color = "gray">
※山本さんがお使いのデータと違い、65歳で分けていません。悪しからず。
</font>

都道府県名から「都府県」を外して短くしたものと、複数の県を地方別にまとめたグループ（異論があるかもしれませんが、お許しを）も作っています。Flourishで可視化する際に見やすくしたかったからです。

In [14]:
# 都道府県コードと人口データ、ネ申状態でない、よいものが見つからなかった。
# JIS X 0401を整形したものと、"https://www.soumu.go.jp/main_content/000701580.xls"の人口データで作った。
# コード順の都道府県名（末尾に都道府県をつけずに短くした形）＋全国。
levels_prefecture_short <- c(
  "北海道", "青森",   "岩手",   "宮城",   "秋田",   "山形",  
  "福島",   "茨城",   "栃木",   "群馬",   "埼玉",   "千葉",  
  "東京",   "神奈川", "新潟",   "富山",   "石川",   "福井",  
  "山梨",   "長野",   "岐阜",   "静岡",   "愛知",   "三重",  
  "滋賀",   "京都",   "大阪",   "兵庫",   "奈良",   "和歌山",
  "鳥取",   "島根",   "岡山",   "広島",   "山口",   "徳島",  
  "香川",   "愛媛",   "高知",   "福岡",   "佐賀",   "長崎",  
  "熊本",   "大分",   "宮崎",   "鹿児島", "沖縄",   "全国" 
)

# 2020年1月1日時点の都道府県別人口。出典は上記。
population_2020 <- c(
   5267762, 1275783, 1235517, 2292385,  985416, 1082296,
   1881981, 2921436, 1965516, 1969439, 7390054, 6319772,
  13834925, 9209442, 2236042, 1055999, 1139612,  780053,
    826579, 2087307, 2032490, 3708556, 7575530, 1813859,
   1420948, 2545899, 8849635, 5549568, 1353837,  954258,
    561175,  679324, 1903627, 2826858, 1369882,  742505,
    981280, 1369131,  709230, 5129841, 823810,  1350769,
   1769880, 1151229, 1095903, 1630146, 1481547, 127138033
)

# 都道府県コードを作る。頭に0がつく文字列型。
JISX0401 <- 1:47 %>%
    str_pad(2, pad = 0)
JISX0401[48] <- "TOTAL/AVE"

# 都・府・県がついていないと、突合に困りそうなので、それも作る。
levels_prefecture <- levels_prefecture_short[1:48] %>% 
    paste0("県")
levels_prefecture[ 1] <- "北海道"
levels_prefecture[13] <- "東京都"
levels_prefecture[26] <- "京都府"
levels_prefecture[27] <- "大阪府"
levels_prefecture[48] <- "全国"

# この４つをまとめてtibbleにする。
pref_pop <- tibble(
    code                = JISX0401, 
    prefecture          = levels_prefecture,
    prefecture_short    = levels_prefecture_short,
    population          = population_2020
)

# 辞書順ではなく、出現順通りのfactor にしてみた。
columns_to_integer <- 'population'
columns_to_factor2 <- c('code', 'prefecture', 'prefecture_short')
pref_pop <- pref_pop %>% 
    mutate(
        across(all_of(columns_to_integer), as.integer),
        across(all_of(columns_to_factor2), fct_inorder)
    )

# 地方別にくくれるようにする。中点があるものは、エラーしないようにバックティックで囲む。
pref_pop <- pref_pop %>% 
    mutate(block = prefecture_short)
pref_pop <- pref_pop %>% 
    mutate(block = fct_collapse(block,
  `北海道・東北` = c("北海道", "青森",   "岩手",   "宮城",   "秋田",   "山形",   "福島"),
  関東           = c("茨城",   "栃木",   "群馬",   "埼玉",   "千葉",   "東京",   "神奈川"),
  甲信越         = c("新潟",   "富山",   "石川",   "福井",   "山梨",   "長野"),
  東海           = c("岐阜",   "静岡",   "愛知",   "三重"),  
  近畿           = c("滋賀",   "京都",   "大阪",   "兵庫",   "奈良",   "和歌山"),
  中国           = c("鳥取",   "島根",   "岡山",   "広島",   "山口"),
  四国           = c("徳島",   "香川",   "愛媛",   "高知"),
  `九州・沖縄`   = c("福岡",   "佐賀",   "長崎",   "熊本",   "大分",   "宮崎",   "鹿児島", "沖縄"),
  全国           = "全国" 
    )
)

head(pref_pop)
summary(pref_pop)
str(pref_pop)

code,prefecture,prefecture_short,population,block
<fct>,<fct>,<fct>,<int>,<fct>
01,北海道,北海道,5267762,北海道・東北
02,青森県,青森,1275783,北海道・東北
03,岩手県,岩手,1235517,北海道・東北
04,宮城県,宮城,2292385,北海道・東北
05,秋田県,秋田,985416,北海道・東北
06,山形県,山形,1082296,北海道・東北


      code      prefecture prefecture_short   population       
 01     : 1   北海道 : 1   北海道 : 1       Min.   :   561175  
 02     : 1   青森県 : 1   青森   : 1       1st Qu.:  1092501  
 03     : 1   岩手県 : 1   岩手   : 1       Median :  1700013  
 04     : 1   宮城県 : 1   宮城   : 1       Mean   :  5297418  
 05     : 1   秋田県 : 1   秋田   : 1       3rd Qu.:  2850502  
 06     : 1   山形県 : 1   山形   : 1       Max.   :127138033  
 (Other):42   (Other):42   (Other):42                          
          block  
 九州・沖縄  :8  
 北海道・東北:7  
 関東        :7  
 甲信越      :6  
 近畿        :6  
 中国        :5  
 (Other)     :9  

tibble [48 × 5] (S3: tbl_df/tbl/data.frame)
 $ code            : Factor w/ 48 levels "01","02","03",..: 1 2 3 4 5 6 7 8 9 10 ...
 $ prefecture      : Factor w/ 48 levels "北海道","青森県",..: 1 2 3 4 5 6 7 8 9 10 ...
 $ prefecture_short: Factor w/ 48 levels "北海道","青森",..: 1 2 3 4 5 6 7 8 9 10 ...
 $ population      : int [1:48] 5267762 1275783 1235517 2292385 985416 1082296 1881981 2921436 1965516 1969439 ...
 $ block           : Factor w/ 9 levels "北海道・東北",..: 1 1 1 1 1 1 1 2 2 2 ...


## 6.感染状況

### 6.1. データを読み込む

感染状況のデータを読み込みます。都道府県ごとに取ってくることもできます。いまは、全国分をまとめて取ってきましょう。

なお、JSON形式ではありますが、接種状況とは違い、改行区切りにはなっていないので、NDJSON形式ではありません。圧縮もかかっていません。

なので、先ほどとは、読み込みの仕方がちょっと違います。

In [15]:
# 内閣官房のページにある、都道府県別の感染者数データ。
# ダウンロードの仕方はちょっと異なる。
target_url_opendata <- "https://opendata.corona.go.jp/api/Covid19JapanAll"
ndjson::validate(target_url_opendata, verbose = TRUE)
opendata_json <- jsonlite::fromJSON(target_url_opendata)
# エラーは出ていないようだ。必要な部分だけを取り出す。
str(opendata_json)
cum_patients <- opendata_json[["itemList"]] %>%
# data.frameなのでtibbleに変換する。
    as_tibble()
head(cum_patients)
# 不要になった元データは消す。
rm(opendata_json)

File: https://opendata.corona.go.jp/api/Covid19JapanAll


[1] TRUE

List of 2
 $ errorInfo:List of 3
  ..$ errorFlag   : chr "0"
  ..$ errorCode   : NULL
  ..$ errorMessage: NULL
 $ itemList :'data.frame':	20774 obs. of  3 variables:
  ..$ date     : chr [1:20774] "2021-07-07" "2021-07-07" "2021-07-07" "2021-07-07" ...
  ..$ name_jp  : chr [1:20774] "北海道" "青森県" "岩手県" "宮城県" ...
  ..$ npatients: chr [1:20774] "41596" "2531" "1746" "9222" ...


date,name_jp,npatients
<chr>,<chr>,<chr>
2021-07-07,北海道,41596
2021-07-07,青森県,2531
2021-07-07,岩手県,1746
2021-07-07,宮城県,9222
2021-07-07,秋田県,918
2021-07-07,山形県,2030


しかし、全国計のデータはまた、別に用意されています。単純に、各県の合算とはいかないからでしょう。

In [16]:
# 同じく内閣官房のページにある、全国計の感染者数データ。
# 手元で計算してみると、上で読み込んだ各都道府県の合算より多い。
target_url_opendata_jpn <-
    "https://data.corona.go.jp/converted-json/covid19japan-npatients.json"
ndjson::validate(target_url_opendata_jpn, verbose = TRUE)
opendata_jpn <- jsonlite::fromJSON(target_url_opendata_jpn)
# データの仕様がよく分からない。もったいない。
# サイトのグラフと見比べると、新規感染者数はadpatientsカラムの数字のようだ。
str(opendata_jpn)
cum_patients_jpn <- opendata_jpn %>% 
    transmute(
        date         = date %>% parse_date(),
        prefecture   = levels_prefecture[48],
        npatients    = npatients %>% as.integer(),
        new_patients = adpatients %>% as.integer()
# tibbleにする。
    ) %>% as_tibble()
head(cum_patients_jpn)

# 不要になった元データは消す。
rm(opendata_jpn)

File: https://data.corona.go.jp/converted-json/covid19japan-npatients.json


[1] TRUE

'data.frame':	444 obs. of  3 variables:
 $ date      : chr  "2020-04-21" "2020-04-22" "2020-04-23" "2020-04-24" ...
 $ npatients : int  11350 11772 12240 12681 13031 13232 13422 13695 13929 14120 ...
 $ adpatients: int  370 420 434 423 346 199 183 272 214 187 ...


date,prefecture,npatients,new_patients
<date>,<chr>,<int>,<int>
2020-04-21,全国,11350,370
2020-04-22,全国,11772,420
2020-04-23,全国,12240,434
2020-04-24,全国,12681,423
2020-04-25,全国,13031,346
2020-04-26,全国,13232,199


### 6.2. 型を変換する

3カラムとも文字列型なので、日付・カテゴリー・整数、にそれぞれ変換します。

全国分のほうは、dateカラムは正しく日付型に、感染者数は両カラムとも整数型になっています。

In [17]:
# 日付型と整数型に変換が必要なカラムを処理。さっきとは別の、シンプルな書き方にしてみた。
cum_patients <- cum_patients %>% 
# transmuteはmutateの変種で、名挙げしないカラムは消してしまう。
    transmute(
        date        = date %>% ymd(),
        prefecture  = name_jp,
        npatients   = npatients %>% parse_integer()
    )
str(cum_patients)
head(cum_patients)
summary(cum_patients)

tibble [20,774 × 3] (S3: tbl_df/tbl/data.frame)
 $ date      : Date[1:20774], format: "2021-07-07" "2021-07-07" ...
 $ prefecture: chr [1:20774] "北海道" "青森県" "岩手県" "宮城県" ...
 $ npatients : int [1:20774] 41596 2531 1746 9222 918 2030 5012 10704 7173 8060 ...


date,prefecture,npatients
<date>,<chr>,<int>
2021-07-07,北海道,41596
2021-07-07,青森県,2531
2021-07-07,岩手県,1746
2021-07-07,宮城県,9222
2021-07-07,秋田県,918
2021-07-07,山形県,2030


      date             prefecture          npatients     
 Min.   :2020-04-22   Length:20774       Min.   :     0  
 1st Qu.:2020-08-10   Class :character   1st Qu.:   178  
 Median :2020-11-28   Mode  :character   Median :   843  
 Mean   :2020-11-28                      Mean   :  5719  
 3rd Qu.:2021-03-19                      3rd Qu.:  3482  
 Max.   :2021-07-07                      Max.   :178356  

### 6.3. 新規感染者数と移動平均を計算する

感染者数は、その日までの累計です。これをもとに、その日に増えた分＝新規感染者数を計算したいのですが、どうすればよいでしょう？

日付順になっているので、まず都道府県別、その中で日付順、というふうに直します。それから、npatientsのカラムの、当日の数字から、前日の数字を引き算したものを、新しいカラムを作って保存します。さらに、曜日要因を取り除くため、7日移動平均も計算します。

いよいよ、松波さんの資料を参考にして下さい。

7日移動平均は、前後でなく過去1週間分を計算しています。最新の日付が上に来るようにしているので、lagではなくleadを使いました。

1列が1日分で、途中の抜けがないかどうか。新規感染者については、数字に変化がない場合はデータなし、という構造だったとしても動作しますが、移動平均のほうはNGです。仮に途中に抜けがある場合は、行ではなく日付（時系列データ）をちゃんと見て処理しないといけません。深入りしませんが、マイナーなパッケージを使うと解決できる場合もあります。[こちら](https://rstudio-pubs-static.s3.amazonaws.com/640203_28880d5e47b44029b9279d8e24385fba.html)を参考にして下さい。

In [18]:
# 都道府県別に小分けした表を対象に、新規感染者と移動平均を計算する準備。即席のファンクションを作った。
calc_diff <- function(nested_df){
    return(
        nested_df %>%
        arrange(desc(date)) %>% 
        mutate(
            new_patients = npatients - dplyr::lead(npatients),
            patients_ma7 = roll_meanl(new_patients, 7, )
        )
    )
}

# 都道府県ごとにnest（下の階層に畳む）する。
patients <- cum_patients %>%
    nest(data_by_pref = c(date, npatients)) %>%
# さっき作ったファンクションを、県単位で当てはめていく
    mutate(data_by_pref = map(data_by_pref, calc_diff)) %>% 
# 計算が終わったので、nestを解除する。
    unnest(cols = data_by_pref)
# 不要なデータを消す。
rm(cum_patients)

# 結果を見てみる。
head(patients)
str(patients)
summary(patients)

prefecture,date,npatients,new_patients,patients_ma7
<chr>,<date>,<int>,<int>,<dbl>
北海道,2021-07-07,41596,67,37.71429
北海道,2021-07-06,41529,34,33.57143
北海道,2021-07-05,41495,29,32.00000
北海道,2021-07-04,41466,39,31.71429
北海道,2021-07-03,41427,30,30.14286
北海道,2021-07-02,41397,26,32.57143


tibble [20,774 × 5] (S3: tbl_df/tbl/data.frame)
 $ prefecture  : chr [1:20774] "北海道" "北海道" "北海道" "北海道" ...
 $ date        : Date[1:20774], format: "2021-07-07" "2021-07-06" ...
 $ npatients   : int [1:20774] 41596 41529 41495 41466 41427 41397 41371 41332 41294 41271 ...
 $ new_patients: int [1:20774] 67 34 29 39 30 26 39 38 23 27 ...
 $ patients_ma7: num [1:20774] 37.7 33.6 32 31.7 30.1 ...


  prefecture             date              npatients       new_patients    
 Length:20774       Min.   :2020-04-22   Min.   :     0   Min.   :-989.00  
 Class :character   1st Qu.:2020-08-10   1st Qu.:   178   1st Qu.:   0.00  
 Mode  :character   Median :2020-11-28   Median :   843   Median :   4.00  
                    Mean   :2020-11-28   Mean   :  5719   Mean   :  38.43  
                    3rd Qu.:2021-03-19   3rd Qu.:  3482   3rd Qu.:  25.00  
                    Max.   :2021-07-07   Max.   :178356   Max.   :2447.00  
                                                          NA's   :47       
  patients_ma7      
 Min.   : -94.2857  
 1st Qu.:   0.5714  
 Median :   5.7143  
 Mean   :  38.6683  
 3rd Qu.:  26.8571  
 Max.   :1812.7143  
 NA's   :329        

前日との差が「マイナス」になっている箇所もあります。抽出してみましょう。

In [19]:
patients %>% filter(
    new_patients < 0
)

prefecture,date,npatients,new_patients,patients_ma7
<chr>,<date>,<int>,<int>,<dbl>
福島県,2021-04-06,2654,-258,25.0000000
栃木県,2020-10-12,444,-1,0.8571429
栃木県,2020-04-25,52,-1,NA
埼玉県,2020-05-30,1003,-1,0.5714286
千葉県,2020-06-05,904,-3,-0.2857143
千葉県,2020-06-02,906,-3,0.2857143
神奈川県,2020-09-23,5452,-989,-94.2857143
神奈川県,2020-05-10,1150,-8,7.0000000
新潟県,2020-09-07,145,-1,0.2857143


感染者数の全国分は、もともと新規のデータがあるので、差分の計算は不要（計算しても、合いません）。7日移動平均のみ計算します。

その上で、各都道府県の感染者数のデータとまとめて、一つの表にします。

In [20]:
# 全国の感染者数については、新規は計算不要。移動平均のみ計算する。
cum_patients_jpn <- cum_patients_jpn %>%
    arrange(desc(date)) %>% 
    mutate(
        patients_ma7 = roll_meanl(new_patients, 7, )
    )
# 先ほどの各都道府県分の表と結合する。
patients <- patients %>%
    bind_rows(cum_patients_jpn)

# 累計のカラムの名前をワクチン接種の表に合わせて直しておく。
patients <- patients %>%
    rename("sum_patients" = npatients)
# 辞書順ではなく、コード順に都道府県をソート可能にしたいので、カテゴリー型に。
patients$prefecture <- parse_factor(patients$prefecture, levels = levels_prefecture)

# 結果を見る。
head(patients)
tail(patients)
str(patients)

# 不要になったデータは消す
rm(cum_patients_jpn)

prefecture,date,sum_patients,new_patients,patients_ma7
<fct>,<date>,<int>,<int>,<dbl>
北海道,2021-07-07,41596,67,37.71429
北海道,2021-07-06,41529,34,33.57143
北海道,2021-07-05,41495,29,32.00000
北海道,2021-07-04,41466,39,31.71429
北海道,2021-07-03,41427,30,30.14286
北海道,2021-07-02,41397,26,32.57143


prefecture,date,sum_patients,new_patients,patients_ma7
<fct>,<date>,<int>,<int>,<dbl>
全国,2020-04-26,13232,199,NA
全国,2020-04-25,13031,346,NA
全国,2020-04-24,12681,423,NA
全国,2020-04-23,12240,434,NA
全国,2020-04-22,11772,420,NA
全国,2020-04-21,11350,370,NA


tibble [21,218 × 5] (S3: tbl_df/tbl/data.frame)
 $ prefecture  : Factor w/ 48 levels "北海道","青森県",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ date        : Date[1:21218], format: "2021-07-07" "2021-07-06" ...
 $ sum_patients: int [1:21218] 41596 41529 41495 41466 41427 41397 41371 41332 41294 41271 ...
 $ new_patients: int [1:21218] 67 34 29 39 30 26 39 38 23 27 ...
 $ patients_ma7: num [1:21218] 37.7 33.6 32 31.7 30.1 ...


## 7.接種状況と感染状況をドッキング

### 7.1. 何をキーに使うか

最初に処理したワクチン接種のデータと、後から処理した感染者数のデータを、ドッキングします。同じ都道府県どうし、同じ日付のものを間違いなく、対応させなくてはなりません。

ポイントは、それぞれのデータで、一意のキーとなるものはどれか、です。それを使って、突き合わせしていきます。

もう一回、vaccinationsとpatientsを見ておきましょう。

In [21]:
head(vaccinations)
tail(vaccinations)

prefecture,date,n_vaccinations,total_vaccinations,cum_vaccinations
<chr>,<date>,<dbl>,<dbl>,<dbl>
01,2021-07-09,20416,1901266,1880850
01,2021-07-08,33336,1880850,1847514
01,2021-07-07,36208,1847514,1811306
01,2021-07-06,40353,1811306,1770953
01,2021-07-05,44455,1770953,1726498
01,2021-07-04,37891,1726498,1688607


prefecture,date,n_vaccinations,total_vaccinations,cum_vaccinations
<chr>,<date>,<dbl>,<dbl>,<dbl>
TOTAL/AVE,2021-04-17,7652,39583,31931
TOTAL/AVE,2021-04-16,9796,31931,22135
TOTAL/AVE,2021-04-15,8492,22135,13643
TOTAL/AVE,2021-04-14,4794,13643,8849
TOTAL/AVE,2021-04-13,4112,8849,4737
TOTAL/AVE,2021-04-12,4737,4737,0


In [22]:
head(patients)
tail(patients)

prefecture,date,sum_patients,new_patients,patients_ma7
<fct>,<date>,<int>,<int>,<dbl>
北海道,2021-07-07,41596,67,37.71429
北海道,2021-07-06,41529,34,33.57143
北海道,2021-07-05,41495,29,32.00000
北海道,2021-07-04,41466,39,31.71429
北海道,2021-07-03,41427,30,30.14286
北海道,2021-07-02,41397,26,32.57143


prefecture,date,sum_patients,new_patients,patients_ma7
<fct>,<date>,<int>,<int>,<dbl>
全国,2020-04-26,13232,199,NA
全国,2020-04-25,13031,346,NA
全国,2020-04-24,12681,423,NA
全国,2020-04-23,12240,434,NA
全国,2020-04-22,11772,420,NA
全国,2020-04-21,11350,370,NA


「prefectureカラムとdateカラムのペア」をひとまとめにしたものが、どちらの表でも、それぞれの行に固有なキーになっていますね。これを生かして、××県の△△日のデータ、と指定すれば、狙ったものが選び出せるわけです。

でも、同じprefectureカラムでも、一方は都道府県コードで、もう一方は都道府県名です。それで、対照表を用意していたのでした。なので、まずは対照表とドッキングさせます。ほかの手順も考えられますが、ここでは、両方の表を対照表とドッキングさせます。


### 7.2. 対照表とjoinする

2つのtibbleを付き合わせて結合するには、joinを使います。

いくつか種類があります。突合できないデータがあったときに、残すか、捨てるか、が違います。きょうは深入りしませんので、ご自身で使う際に調べて下さい。NA値を「どれも同じもの」と見なすか否か、伝統的なデータベースとRでは、デフォルトの挙動が違ったりします。

patientsには、県名のカラムprefectureがあります。これをキーにして、5.2.で作った対照表pref_popとjoinします。対照表にもprefectureカラムがあり、やはり一意のキーになっています。


In [23]:
# patientsとvaccinationsを結合する。
# そのために、この２つの表の「都道府県」の対照表を使う。
# patientsには都道府県名のカラムがあるので、それをキーに対照表と結合。
patients <- patients %>%
# prefecture以外は畳む。
    nest(data = -prefecture) %>%
# prefectureカラムをキーにして突合する。
    left_join(pref_pop, by = "prefecture") %>% 
# ネストを戻す。
unnest(cols = data)

# こうなった
head(patients)
tail(patients)

prefecture,date,sum_patients,new_patients,patients_ma7,code,prefecture_short,population,block
<fct>,<date>,<int>,<int>,<dbl>,<fct>,<fct>,<int>,<fct>
北海道,2021-07-07,41596,67,37.71429,01,北海道,5267762,北海道・東北
北海道,2021-07-06,41529,34,33.57143,01,北海道,5267762,北海道・東北
北海道,2021-07-05,41495,29,32.00000,01,北海道,5267762,北海道・東北
北海道,2021-07-04,41466,39,31.71429,01,北海道,5267762,北海道・東北
北海道,2021-07-03,41427,30,30.14286,01,北海道,5267762,北海道・東北
北海道,2021-07-02,41397,26,32.57143,01,北海道,5267762,北海道・東北


prefecture,date,sum_patients,new_patients,patients_ma7,code,prefecture_short,population,block
<fct>,<date>,<int>,<int>,<dbl>,<fct>,<fct>,<int>,<fct>
全国,2020-04-26,13232,199,NA,TOTAL/AVE,全国,127138033,全国
全国,2020-04-25,13031,346,NA,TOTAL/AVE,全国,127138033,全国
全国,2020-04-24,12681,423,NA,TOTAL/AVE,全国,127138033,全国
全国,2020-04-23,12240,434,NA,TOTAL/AVE,全国,127138033,全国
全国,2020-04-22,11772,420,NA,TOTAL/AVE,全国,127138033,全国
全国,2020-04-21,11350,370,NA,TOTAL/AVE,全国,127138033,全国


vaccinationsにも、prefectureというカラムはあるのですが、中身は県名ではなく、都道府県コードです。これをキーにjoinします。相手のpref_popに合わせて、カラム名はあらかじめcodeに直しておきます。pref_popのcodeカラムも、一意のキーになっています。

In [24]:
# vaccinationsはコードをキーに対照表と結合する。
vaccinations <- vaccinations %>% 
# カラム名を揃えてからjoin。ここでカテゴリー型に直しておく。
    mutate(
        "code" = parse_factor(prefecture, levels = JISX0401)
    ) %>%
    select(-prefecture) %>%
# code以外は畳む
    nest(data = -code) %>%
# codeカラムをキーにして突合する。
    left_join(pref_pop, by = "code") %>%
# ネストを戻す。
    unnest(cols = data)

# こうなった
head(vaccinations)
tail(vaccinations)

code,date,n_vaccinations,total_vaccinations,cum_vaccinations,prefecture,prefecture_short,population,block
<fct>,<date>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<int>,<fct>
01,2021-07-09,20416,1901266,1880850,北海道,北海道,5267762,北海道・東北
01,2021-07-08,33336,1880850,1847514,北海道,北海道,5267762,北海道・東北
01,2021-07-07,36208,1847514,1811306,北海道,北海道,5267762,北海道・東北
01,2021-07-06,40353,1811306,1770953,北海道,北海道,5267762,北海道・東北
01,2021-07-05,44455,1770953,1726498,北海道,北海道,5267762,北海道・東北
01,2021-07-04,37891,1726498,1688607,北海道,北海道,5267762,北海道・東北


code,date,n_vaccinations,total_vaccinations,cum_vaccinations,prefecture,prefecture_short,population,block
<fct>,<date>,<dbl>,<dbl>,<dbl>,<fct>,<fct>,<int>,<fct>
TOTAL/AVE,2021-04-17,7652,39583,31931,全国,全国,127138033,全国
TOTAL/AVE,2021-04-16,9796,31931,22135,全国,全国,127138033,全国
TOTAL/AVE,2021-04-15,8492,22135,13643,全国,全国,127138033,全国
TOTAL/AVE,2021-04-14,4794,13643,8849,全国,全国,127138033,全国
TOTAL/AVE,2021-04-13,4112,8849,4737,全国,全国,127138033,全国
TOTAL/AVE,2021-04-12,4737,4737,0,全国,全国,127138033,全国


### 7.3. 接種状況と感染者数をJOINする

これで、接種状況と感染者数をドッキングさせる準備が整いました。都道府県だけ、日付だけ、では一意のキーになりませんが、都道府県と日付のペアで見れば、両方を結びつける際のキーになります。

なお、ほかにも共通のカラムはありますが、これは、同じpref_popとそれぞれの表を先ほどjoinしたからです。長短の県名・県コード・日付のほかに地域もキーにして突合しています、とメッセージが出ていますが、差し支えありません。今回は、キーを指定しないでjoinを指示したので、両方に同じカラム名のデータがあれば、それをキーに使おうとしてくれているわけです。


In [25]:
# vaccinationとpatientsを付き合わせる。
# left_joinは、左のデータにあるものは、右に突合相手がいなくても残す。
# なので、これはワクチン接種が始まる前の感染者データも保存する。
joined_data <- left_join(patients, vaccinations) %>%
# カラムを並べ替えて見やすくする
    relocate(date, code, prefecture, prefecture_short, block, population)
# こうなった。
head(joined_data)
tail(joined_data)
summary(joined_data)
str(joined_data)

Joining, by = c("prefecture", "date", "code", "prefecture_short", "population", "block")



date,code,prefecture,prefecture_short,block,population,sum_patients,new_patients,patients_ma7,n_vaccinations,total_vaccinations,cum_vaccinations
<date>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
2021-07-07,01,北海道,北海道,北海道・東北,5267762,41596,67,37.71429,36208,1847514,1811306
2021-07-06,01,北海道,北海道,北海道・東北,5267762,41529,34,33.57143,40353,1811306,1770953
2021-07-05,01,北海道,北海道,北海道・東北,5267762,41495,29,32.00000,44455,1770953,1726498
2021-07-04,01,北海道,北海道,北海道・東北,5267762,41466,39,31.71429,37891,1726498,1688607
2021-07-03,01,北海道,北海道,北海道・東北,5267762,41427,30,30.14286,36366,1688607,1652241
2021-07-02,01,北海道,北海道,北海道・東北,5267762,41397,26,32.57143,33505,1652241,1618736


date,code,prefecture,prefecture_short,block,population,sum_patients,new_patients,patients_ma7,n_vaccinations,total_vaccinations,cum_vaccinations
<date>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
2020-04-26,TOTAL/AVE,全国,全国,全国,127138033,13232,199,NA,NA,NA,NA
2020-04-25,TOTAL/AVE,全国,全国,全国,127138033,13031,346,NA,NA,NA,NA
2020-04-24,TOTAL/AVE,全国,全国,全国,127138033,12681,423,NA,NA,NA,NA
2020-04-23,TOTAL/AVE,全国,全国,全国,127138033,12240,434,NA,NA,NA,NA
2020-04-22,TOTAL/AVE,全国,全国,全国,127138033,11772,420,NA,NA,NA,NA
2020-04-21,TOTAL/AVE,全国,全国,全国,127138033,11350,370,NA,NA,NA,NA


      date                   code         prefecture    prefecture_short
 Min.   :2020-04-21   TOTAL/AVE:  444   全国   :  444   全国   :  444   
 1st Qu.:2020-08-10   01       :  442   北海道 :  442   北海道 :  442   
 Median :2020-11-28   02       :  442   青森県 :  442   青森   :  442   
 Mean   :2020-11-28   03       :  442   岩手県 :  442   岩手   :  442   
 3rd Qu.:2021-03-19   04       :  442   宮城県 :  442   宮城   :  442   
 Max.   :2021-07-08   05       :  442   秋田県 :  442   秋田   :  442   
                      (Other)  :18564   (Other):18564   (Other):18564   
          block        population         sum_patients     new_patients    
 九州・沖縄  :3536   Min.   :   561175   Min.   :     0   Min.   :-989.00  
 北海道・東北:3094   1st Qu.:  1095903   1st Qu.:   184   1st Qu.:   0.00  
 関東        :3094   Median :  1769880   Median :   893   Median :   5.00  
 甲信越      :2652   Mean   :  5308903   Mean   : 11240   Mean   :  75.17  
 近畿        :2652   3rd Qu.:  2921436   3rd Qu.:  3905   3rd Qu.:  28.00  
 中国     

tibble [21,218 × 12] (S3: tbl_df/tbl/data.frame)
 $ date              : Date[1:21218], format: "2021-07-07" "2021-07-06" ...
 $ code              : Factor w/ 48 levels "01","02","03",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ prefecture        : Factor w/ 48 levels "北海道","青森県",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ prefecture_short  : Factor w/ 48 levels "北海道","青森",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ block             : Factor w/ 9 levels "北海道・東北",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ population        : int [1:21218] 5267762 5267762 5267762 5267762 5267762 5267762 5267762 5267762 5267762 5267762 ...
 $ sum_patients      : int [1:21218] 41596 41529 41495 41466 41427 41397 41371 41332 41294 41271 ...
 $ new_patients      : int [1:21218] 67 34 29 39 30 26 39 38 23 27 ...
 $ patients_ma7      : num [1:21218] 37.7 33.6 32 31.7 30.1 ...
 $ n_vaccinations    : num [1:21218] 36208 40353 44455 37891 36366 ...
 $ total_vaccinations: num [1:21218] 1847514 1811306 1770953 1726498 1688607 ...
 $ cum_vaccinations  : num [1:21218]


感染者数のデータは2020年からありますが、ワクチン接種のほうは2021年4月12日以降の分しかありません（県によっては、さらに欠けています）。なので、両方が揃っている期間だけに絞りましょう。

In [26]:
joined_data <- joined_data %>%
    filter(date >= ymd("2021-4-12"))
tail(joined_data)
summary(joined_data)

date,code,prefecture,prefecture_short,block,population,sum_patients,new_patients,patients_ma7,n_vaccinations,total_vaccinations,cum_vaccinations
<date>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
2021-04-17,TOTAL/AVE,全国,全国,全国,127138033,527247,4722,3772.714,7652,39583,31931
2021-04-16,TOTAL/AVE,全国,全国,全国,127138033,522647,4519,3616.714,9796,31931,22135
2021-04-15,TOTAL/AVE,全国,全国,全国,127138033,518187,4570,3466.429,8492,22135,13643
2021-04-14,TOTAL/AVE,全国,全国,全国,127138033,513569,4302,3300.000,4794,13643,8849
2021-04-13,TOTAL/AVE,全国,全国,全国,127138033,509627,3444,3179.143,4112,8849,4737
2021-04-12,TOTAL/AVE,全国,全国,全国,127138033,506272,2094,3063.571,4737,4737,0


      date                   code        prefecture   prefecture_short
 Min.   :2021-04-12   TOTAL/AVE:  88   全国   :  88   全国   :  88    
 1st Qu.:2021-05-03   01       :  87   北海道 :  87   北海道 :  87    
 Median :2021-05-25   02       :  87   青森県 :  87   青森   :  87    
 Mean   :2021-05-25   03       :  87   岩手県 :  87   岩手   :  87    
 3rd Qu.:2021-06-16   04       :  87   宮城県 :  87   宮城   :  87    
 Max.   :2021-07-08   05       :  87   秋田県 :  87   秋田   :  87    
                      (Other)  :3654   (Other):3654   (Other):3654    
          block       population         sum_patients     new_patients   
 九州・沖縄  :696   Min.   :   561175   Min.   :   292   Min.   :   0.0  
 北海道・東北:609   1st Qu.:  1095903   1st Qu.:  2017   1st Qu.:   6.0  
 関東        :609   Median :  1769880   Median :  4288   Median :  22.0  
 甲信越      :522   Mean   :  5326587   Mean   : 28993   Mean   : 145.8  
 近畿        :522   3rd Qu.:  2921436   3rd Qu.: 11397   3rd Qu.:  67.0  
 中国        :435   Max.   :127138033 

### 7.4. このNA値は何か

この期間に絞っても、NA（Not Available）値がまだ残っています。これは、データが欠けていた部分です。
もともとがJSON、取り込んだ後も縦持ち形式だったので、気づきにくいですよね。でも、横持ちに変換したときに、気になった方もいたはずです。

これが、当該の県ではその日に接種がなかった、ということを意味するのであれば（要確認）、0に置き換えてよいわけです。

In [27]:
# 全カラムのデータが揃っていない行だけを抽出する。
rows_w_na <- !complete.cases(joined_data)
joined_data[rows_w_na, ]

date,code,prefecture,prefecture_short,block,population,sum_patients,new_patients,patients_ma7,n_vaccinations,total_vaccinations,cum_vaccinations
<date>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
2021-04-18,02,青森県,青森,北海道・東北,1275783,1305,6,20.2857143,NA,NA,NA
2021-04-18,03,岩手県,岩手,北海道・東北,1235517,779,3,3.8571429,NA,NA,NA
2021-04-18,04,宮城県,宮城,北海道・東北,2292385,7580,28,59.4285714,NA,NA,NA
2021-05-05,05,秋田県,秋田,北海道・東北,985416,536,19,15.1428571,NA,NA,NA
2021-05-04,05,秋田県,秋田,北海道・東北,985416,517,11,13.1428571,NA,NA,NA
2021-05-03,05,秋田県,秋田,北海道・東北,985416,506,14,12.7142857,NA,NA,NA
2021-04-18,07,福島県,福島,北海道・東北,1881981,2985,37,25.5714286,NA,NA,NA
2021-04-17,07,福島県,福島,北海道・東北,1881981,2948,21,22.4285714,NA,NA,NA
2021-04-18,09,栃木県,栃木,関東,1965516,5068,11,20.1428571,NA,NA,NA


In [28]:
# NAは貴重な情報なので、インプレイスで置換せずに、残しておく。
joined_data_w0 <- joined_data
# こちらで、0に置き換えたものを作る。
joined_data_w0[is.na(joined_data)] <- 0
# 上と同じ行だけを表示する。
joined_data_w0[rows_w_na, ]

date,code,prefecture,prefecture_short,block,population,sum_patients,new_patients,patients_ma7,n_vaccinations,total_vaccinations,cum_vaccinations
<date>,<fct>,<fct>,<fct>,<fct>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
2021-04-18,02,青森県,青森,北海道・東北,1275783,1305,6,20.2857143,0,0,0
2021-04-18,03,岩手県,岩手,北海道・東北,1235517,779,3,3.8571429,0,0,0
2021-04-18,04,宮城県,宮城,北海道・東北,2292385,7580,28,59.4285714,0,0,0
2021-05-05,05,秋田県,秋田,北海道・東北,985416,536,19,15.1428571,0,0,0
2021-05-04,05,秋田県,秋田,北海道・東北,985416,517,11,13.1428571,0,0,0
2021-05-03,05,秋田県,秋田,北海道・東北,985416,506,14,12.7142857,0,0,0
2021-04-18,07,福島県,福島,北海道・東北,1881981,2985,37,25.5714286,0,0,0
2021-04-17,07,福島県,福島,北海道・東北,1881981,2948,21,22.4285714,0,0,0
2021-04-18,09,栃木県,栃木,関東,1965516,5068,11,20.1428571,0,0,0


### 7.5. 人口比にする

都道府県どうしの比較をするために、実数だけでなく、人口比も知りたくなります。
各県の接種と感染の数値データを、その県の人口で割って比率を出します。ここでは、定数を掛けて、人口10万人あたりの数にしています。

In [29]:
# 日付と数値データを県ごとに畳む。
joined_data_w0 <-
    joined_data_w0 %>%
# 県ごとの小分けtibbleになる部分
    nest(data = c(
        date,
        new_patients,
        patients_ma7,
        sum_patients,
        n_vaccinations,
        total_vaccinations,
        cum_vaccinations
        )
    ) 

# 県ごとのデータを、その県の人口で割って比率を計算するファンクション。
# 人口10万人あたり、にしている。rをつけたカラムが人口比。
func_ratio <- function(nested_df, divisor){
    const= 100000
    return(
        nested_df %>% mutate(
            "r_new_patients"        = new_patients          / divisor * const,
            "r_patients_ma7"        = patients_ma7          / divisor * const,
            "r_sum_patients"        = sum_patients          / divisor * const,
            "r_n_vaccinations"      = n_vaccinations        / divisor * const,
            "r_total_vaccinations"  = total_vaccinations    / divisor * const,
            "r_cum_vaccinations"    = cum_vaccinations      / divisor * const
        ) %>%
# 念のため、日付降順（最新日付が上）に来るようにソートしておく。
    arrange(desc(date))
    )
}
# これに限らずだが、とくにこのファンクション、松波さんのアドバイスに深謝！

joined_data_ratio <- joined_data_w0 %>% 
    mutate(
# さっきのファンクションに働いてもらう。
        data = map2(data, population, func_ratio)
    ) %>%
# 念のため、県コードの順にソートしておく。
    arrange(prefecture)

# ネストを解除する。
joined_data_ratio <- joined_data_ratio %>% 
    unnest(cols = data)

# こうなった。
str(joined_data_ratio)
head(joined_data_ratio)
tail(joined_data_ratio)
summary(joined_data_ratio)

tibble [4,177 × 18] (S3: tbl_df/tbl/data.frame)
 $ code                : Factor w/ 48 levels "01","02","03",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ prefecture          : Factor w/ 48 levels "北海道","青森県",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ prefecture_short    : Factor w/ 48 levels "北海道","青森",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ block               : Factor w/ 9 levels "北海道・東北",..: 1 1 1 1 1 1 1 1 1 1 ...
 $ population          : int [1:4177] 5267762 5267762 5267762 5267762 5267762 5267762 5267762 5267762 5267762 5267762 ...
 $ date                : Date[1:4177], format: "2021-07-07" "2021-07-06" ...
 $ new_patients        : int [1:4177] 67 34 29 39 30 26 39 38 23 27 ...
 $ patients_ma7        : num [1:4177] 37.7 33.6 32 31.7 30.1 ...
 $ sum_patients        : int [1:4177] 41596 41529 41495 41466 41427 41397 41371 41332 41294 41271 ...
 $ n_vaccinations      : num [1:4177] 36208 40353 44455 37891 36366 ...
 $ total_vaccinations  : num [1:4177] 1847514 1811306 1770953 1726498 1688607 ...
 $ cum_vaccinations   

code,prefecture,prefecture_short,block,population,date,new_patients,patients_ma7,sum_patients,n_vaccinations,total_vaccinations,cum_vaccinations,r_new_patients,r_patients_ma7,r_sum_patients,r_n_vaccinations,r_total_vaccinations,r_cum_vaccinations
<fct>,<fct>,<fct>,<fct>,<int>,<date>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
01,北海道,北海道,北海道・東北,5267762,2021-07-07,67,37.71429,41596,36208,1847514,1811306,1.2718874,0.7159451,789.6332,687.3507,35072.09,34384.73
01,北海道,北海道,北海道・東北,5267762,2021-07-06,34,33.57143,41529,40353,1811306,1770953,0.6454354,0.6372996,788.3614,766.0369,34384.73,33618.70
01,北海道,北海道,北海道・東北,5267762,2021-07-05,29,32.00000,41495,44455,1770953,1726498,0.5505184,0.6074686,787.7159,843.9068,33618.70,32774.79
01,北海道,北海道,北海道・東北,5267762,2021-07-04,39,31.71429,41466,37891,1726498,1688607,0.7403524,0.6020448,787.1654,719.2998,32774.79,32055.49
01,北海道,北海道,北海道・東北,5267762,2021-07-03,30,30.14286,41427,36366,1688607,1652241,0.5695018,0.5722137,786.4251,690.3501,32055.49,31365.14
01,北海道,北海道,北海道・東北,5267762,2021-07-02,26,32.57143,41397,33505,1652241,1618736,0.4935682,0.6183163,785.8555,636.0386,31365.14,30729.10


code,prefecture,prefecture_short,block,population,date,new_patients,patients_ma7,sum_patients,n_vaccinations,total_vaccinations,cum_vaccinations,r_new_patients,r_patients_ma7,r_sum_patients,r_n_vaccinations,r_total_vaccinations,r_cum_vaccinations
<fct>,<fct>,<fct>,<fct>,<int>,<date>,<int>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
TOTAL/AVE,全国,全国,全国,127138033,2021-04-17,4722,3772.714,527247,7652,39583,31931,3.714074,2.967416,414.7044,6.018655,31.133878,25.115223
TOTAL/AVE,全国,全国,全国,127138033,2021-04-16,4519,3616.714,522647,9796,31931,22135,3.554405,2.844715,411.0863,7.705011,25.115223,17.410211
TOTAL/AVE,全国,全国,全国,127138033,2021-04-15,4570,3466.429,518187,8492,22135,13643,3.594518,2.726508,407.5783,6.679355,17.410211,10.730857
TOTAL/AVE,全国,全国,全国,127138033,2021-04-14,4302,3300.000,513569,4794,13643,8849,3.383724,2.595604,403.9460,3.770705,10.730857,6.960152
TOTAL/AVE,全国,全国,全国,127138033,2021-04-13,3444,3179.143,509627,4112,8849,4737,2.708867,2.500544,400.8454,3.234280,6.960152,3.725872
TOTAL/AVE,全国,全国,全国,127138033,2021-04-12,2094,3063.571,506272,4737,4737,0,1.647029,2.409642,398.2066,3.725872,3.725872,0.000000


        code        prefecture   prefecture_short          block    
 TOTAL/AVE:  88   全国   :  88   全国   :  88     九州・沖縄  :696  
 01       :  87   北海道 :  87   北海道 :  87     北海道・東北:609  
 02       :  87   青森県 :  87   青森   :  87     関東        :609  
 03       :  87   岩手県 :  87   岩手   :  87     甲信越      :522  
 04       :  87   宮城県 :  87   宮城   :  87     近畿        :522  
 05       :  87   秋田県 :  87   秋田   :  87     中国        :435  
 (Other)  :3654   (Other):3654   (Other):3654     (Other)     :784  
   population             date             new_patients     patients_ma7     
 Min.   :   561175   Min.   :2021-04-12   Min.   :   0.0   Min.   : -13.571  
 1st Qu.:  1095903   1st Qu.:2021-05-03   1st Qu.:   6.0   1st Qu.:   8.286  
 Median :  1769880   Median :2021-05-25   Median :  22.0   Median :  25.000  
 Mean   :  5326587   Mean   :2021-05-25   Mean   : 145.8   Mean   : 147.953  
 3rd Qu.:  2921436   3rd Qu.:2021-06-16   3rd Qu.:  67.0   3rd Qu.:  71.429  
 Max.   :127138033   Max.   :2

In [30]:
# 不要になったデータは消す。
rm(vaccinations, patients, rows_w_na, pref_pop, joined_data)

## 8. CSVを書き出す


最後に、処理したデータをcsv（コンマ区切り形式）のファイルに書き出して、終了です。書き出す前に、あらかじめrenameでカラム名を日本語に直してもよいでしょう。

Colabの場合は、画面左端の上から４番目あたりにある、ファイルのアイコンを押すと、「processed_data.csv」というファイルができあがっているのが見えるはずです。右クリック、または縦に３つ並んだ点のマークを押すと、ダウンロードが選べます。ローカルなら、お使いのPCに直接セーブされます。

csvは、開き方によっては、文字化けすることがあります。エンコードを調べてみて下さい。S-JISで開こうとして化けていたら、UTF-8を指定して下さい。


In [31]:
write_csv(joined_data_ratio, "processed_data.csv")

あとは、このデータをFlourishに渡してやるだけです。

お疲れさまでした。